In [1]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime

from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


# Data

In [2]:
df = pd.read_csv('./../final_dfs/for_modeling/df_final_date_wide_2007.csv', converters={'ags2': str, 'ags5': str} )
df.set_index('ags5', drop=True, inplace=True)
df = df.drop("Unnamed: 0", axis=1)
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df[(df['date'] >= '2010-01-01') & (df['date'] < '2020-01-01')]

In [3]:
# select only column for which all values are avaliable
df = df[['date', 
        'number_of_company_deletions', 
        'number_of_company_liquidations', 
        'number_of_start_ups',
        'unemployment_rate', 
        'registerd_jobs',
        'employees_social_security_at_work',
        'employees_social_security_at_residence']]

In [4]:
# rename date and unemployment_rate for Prophet function 
df = df.rename(columns={'date': 'ds', 'unemployment_rate': 'y'})

In [5]:
df

,ds,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,y,registerd_jobs,employees_social_security_at_work,employees_social_security_at_residence
ags5,,,,,,,,
1001,2010-01-01,14.0,8.0,9.0,13.7,436,37563,24746
1001,2010-02-01,12.0,2.0,8.0,14.1,459,37553,24702
1001,2010-03-01,7.0,6.0,14.0,13.6,407,37707,24903
1001,2010-04-01,11.0,12.0,15.0,13.1,435,37873,25125
1001,2010-05-01,15.0,5.0,10.0,12.5,438,37947,25228
...,...,...,...,...,...,...,...,...
16077,2019-08-01,3.0,3.0,2.0,7.0,990,27980,33983
16077,2019-09-01,1.0,1.0,3.0,6.5,984,28228,34292
16077,2019-10-01,4.0,1.0,3.0,6.5,886,28190,34301


# 401 Model for each kreis

In [33]:
# one-step forecast
def Prophet_forecast(data, config):
    g, sps, ys, sm = config
    # define model
    model = Prophet(growth = g, seasonality_prior_scale = sps, yearly_seasonality = ys, seasonality_mode = sm)
    # fit model
    model_fit = model.fit(data)
    # make one step forecast
    future = model_fit.make_future_dataframe(periods=12, freq='MS')
    res = model_fit.predict(future)
    res.to_csv(f"runs_res/{g}_{sps}_{ys}_{sm}")
    return res.iloc[-12:][["ds","yhat"]]

In [34]:
df_ags5 = df.filter(regex='1001', axis=0)
config = ['linear', 15.0, True, 'additive']
Prophet_forecast(df_ags5, config)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat
120,2020-01-01,8.185825
121,2020-02-01,8.125500
122,2020-03-01,7.934951
123,2020-04-01,7.882228
124,2020-05-01,7.326174
125,2020-06-01,7.301220
126,2020-07-01,7.596089
127,2020-08-01,7.570870
128,2020-09-01,7.219900
129,2020-10-01,7.082717


In [32]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [31]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [30]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model and make forecast for history
    yhat = Prophet_forecast(train, cfg)
    # estimate prediction error
    error = measure_rmse(test['y'], yhat['yhat'])
    return error

In [100]:
def run_model(data, n_test): 
    g = 'linear'
    sps = 10.0
    ys = True
    sm = 'multiplicative'
        with mlflow.start_run():
            params = [g, sps, ys, sm]

            ags5 = list(set(df.index.values.tolist()))
            rmse = []
            for i in ags5:
                df_ags5 = df.filter(regex=i, axis=0)
                current_rmse = walk_forward_validation(df_ags5, n_test, params)
                print(current_rmse)
                rmse.append(current_rmse)

            # Log params
            params = {"growth":g, "sps":sps, "ys":ys, "sm":sm}

            mlflow.log_params(params)

            # Log metrics
            mlflow.log_metric('rmse', np.mean(rmse))
            mlflow.log_metric('mse', np.mean(rmse)**2)

In [ ]:
run_model(df, 12)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.28422807679277473


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.24569282055257768


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

0.10887362732525067


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.8947878710450985


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.2819945112840591


In [20]:
np.mean([10.0, 15.0])

12.5

In [9]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
            # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
#     if result is not None:
#         print(' > Model[%s] %.3f' % (key, result))
        
    print('.', end='')
    return (key, result)

----------
!mlflow experiments create --experiment-name 401ModelsTemp

Created experiment '401ModelsTemp' with id 6

---------

In [ ]:
# data split
n_test = 12
# model config
cfg = [('linear'), (10.0)]
# train 401 models
print("Training", end=' ')
failed_ags5 = []
rmse_scores = []

with mlflow.start_run(experiment_id=6):
    
    for ags5, df_for_kreis in df_y_grouped:
        
        data = list(df_for_kreis.sort_values('ds').drop(['ds'], axis=1).copy()['y'])
        
        try: 
        
            score = score_model(data, n_test, cfg)

            if score is None:
                raise Exception("No Score")
            
            rmse_scores.append(score[1])
            print('.', end='')
        except:
            failed_ags5.append(ags5)
            print('!', end='')
            
    params = {}
    params['p'], params['d'], params['q'] = cfg[0]
    params['s_p'], params['s_d'], params['s_q'], params['m'] = cfg[1]
    params['t'] = cfg[2]
    
    net_rmse = np.mean([rmse**2 for rmse in rmse_scores])**(0.5)
    
    
    mlflow.log_params(params)
    mlflow.log_metric('net_rmse', net_rmse)
      
    

print('\ndone')
print("Failed ags5:", failed_ags5)

Training ........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
rmse_scores

In [ ]:
params

In [37]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [38]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
480,2020-11-26
481,2020-11-27
482,2020-11-28
483,2020-11-29
484,2020-11-30


In [42]:
Prophet?